#  DATA VISUALIZATION PROJECT

## IMPORT LIBRARIES 

In [ ]:
!pip install psycopg2

In [10]:
!pip install -U simplejson

Requirement already up-to-date: simplejson in c:\datascience\anaconda\software\lib\site-packages (3.17.6)


In [13]:

import json
import re
import time
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import psycopg2
from psycopg2.errors import UniqueViolation
from psycopg2.errors import SyntaxError
from sqlalchemy import create_engine
import ssl
from simplejson import JSONDecodeError
import multiprocess
from multiprocess import Pool, cpu_count
from tqdm import tqdm

## FUNCTIONS

In [7]:
def getcomp(year_comp, pag=1):
    '''
    year_comp is the year when competition was done

    Its return is a object soup. It´s a kind of html conteiner based an answer from server
    or can be a integer -1 when there is no information for that year


    '''  
    ssl._create_default_https_context = ssl._create_unverified_context
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings?type=2&year={year_comp}&sortBy=score&page={pag}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('h2', attrs = {'class' : 'no-results'})
    if len(list_results) > 0:
        return -1
    return [soup, pag]


def atualiza_prova_compet(cursor, cod_prova, cod_competicao, score_prova_competicao,data_prova_competicao):
    try: 
        print(f'Inserindo competicao {cod_competicao} e prova {cod_prova}')
        cursor.execute(f"INSERT INTO tb_prova_competicao (cod_prova, cod_competicao, score_prova_competicao,data_prova_competicao)" \
                       f"VALUES ({cod_prova}, {cod_competicao[0]}, {score_prova_competicao},'{data_prova_competicao}')")

    except UniqueViolation:
        print ("prova ja existe na competicao")
                    
def atualiza_atleta_prova_competicao(cursor,cod_prova, cod_competicao, lista_tb_atleta_competicao):
    cursor.execute(f"select cod_prova_competicao from tb_prova_competicao where cod_prova = {cod_prova} and cod_competicao = {cod_competicao[0]}")
    cod_prova_competicao = cursor.fetchone()[0] 
    for x in lista_tb_atleta_competicao:
        try:
            print(f'Inserindo atleta {x[0]}')
            cursor.execute(f"INSERT INTO tb_atleta_prova_competicao (cod_atleta, cod_prova_competicao, result, score)" \
                         f"VALUES ({x[0]}, {cod_prova_competicao}, '{x[1]}', {x[2]})")
        except UniqueViolation:
            print ("atleta ja existe na prova")
            
            
            
            
def atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao):
   
    for x in lista_tb_atleta_competicao:
        try:
            print(f'Inserindo atleta na tb_atleta {x[0]}')
            cursor.execute(f"INSERT INTO tb_atleta (cod_atleta)" \
                         f"VALUES ({x[0]})")
        except UniqueViolation:
            print ("atleta ja existe na tb_atleta")
            
            
            
def update_atleta(cursor,cod_atleta,url_atleta,pais_atleta,nome_atleta):
   
        
            cursor.execute(f"UPDATE tb_atleta SET url_atleta = '{url_atleta}', " \
                         f"pais_atleta = '{pais_atleta}', nome_atleta = '{nome_atleta}' WHERE cod_atleta = {cod_atleta}")
            print (f"atleta {cod_atleta} atualizado")
            
def update_wheter(cursor,data_prova_competicao, pais_competicao, cidade_competicao, json_resp):
    
        try:
            print(f'Inserindo na Tb_wheter -> {data_prova_competicao} {pais_competicao} {cidade_competicao}')
            cursor.execute(f"INSERT INTO tb_wheter (data_prova_competicao, pais_competicao, cidade_competicao, info_wheter)" \
                         f"VALUES ('{data_prova_competicao}', '{pais_competicao}', '{cidade_competicao}', '{json_resp}')")
        except UniqueViolation:
            print ("condições climaticas já existem na base")
            
            
            
def connect_db():
    connection = psycopg2.connect(user="postgres",
                                  password="jujuba11",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db_atletismo",
                                  options="-c search_path=db_atl")


    connection.autocommit = True
    return connection.cursor()

  


## Insertions on tb_competicao

In [ ]:
connection = psycopg2.connect(user="postgres",
                                  password="jujuba11",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db_atletismo",
                                  options="-c search_path=db_atl")


connection.autocommit = True
cursor = connection.cursor()
contador = 0
list_results = []
year= 2008
soup = getcomp(year,1)[0]
pagina = getcomp(year,1)[1]   
last_page = int(soup.find_all('a', attrs = {'class' : 'btn--pagination btn--pag-last pag-control'})[0]['data-page'])
list_competicao = soup.find_all('tbody')[0].find_all('tr')

if last_page > 0:
    for pagina_int in range(last_page):
        pagina_int += 1
        soup = getcomp(year,pagina_int)[0]
        pagina = getcomp(year,pagina_int)[1]   
        list_competicao = soup.find_all('tbody')[0].find_all('tr')
    
        time.sleep(5)

        for line in range(len(list_competicao)-1):
            try:
                cod_competicao = re.findall(r'\d+',list_competicao[line]['data-href'])[0]
                place = list_competicao[line].find_all('td')[0].text.strip()
                #  competition = list_competicao[line].find_all('td')[1].text.strip()
            
                tamanho = len(list_competicao[line].find_all('td')[1].text.strip().split(","))
            
                lista_nome_competicao = list_competicao[line].find_all('td')[1].text.strip().split(",")
                nome_competicao = " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[0].strip()))
                cidade_competicao =  " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[len(lista_nome_competicao)-1].strip()))
                   
         
                country = list_competicao[line].find_all('td')[2].text.strip()
                Start_date = list_competicao[line].find_all('td')[3].text.strip()
                end_date = list_competicao[line].find_all('td')[4].text.strip()
                part_score = list_competicao[line].find_all('td')[5].text.strip()
                ps_place = list_competicao[line].find_all('td')[6].text.strip()
                result_score = list_competicao[line].find_all('td')[7].text.strip()
                rs_place = list_competicao[line].find_all('td')[8].text.strip()
       
                if pagina == 1:
                   comp_score = list_competicao[line].find_all('td')[10].text.strip()
                else:
                    comp_score = list_competicao[line].find_all('td')[9].text.strip()
             
                list_results.append((cod_competicao, nome_competicao, comp_score, part_score, result_score, Start_date, end_date,
                                  cidade_competicao, country, ps_place, rs_place ))
            
            
                sql_update = f"INSERT INTO tb_competicao (cod_competicao, nome_competição, score_competicao," \
                    f"score_participacao_competicao, score_result_competicao, data_inicial_competicao," \
                    f"data_final_competicao, cidade_competicao, pais_competicao," \
                    f"ps_place_competicao, rs_place_competicao)" \
                    f" VALUES ({cod_competicao},'{nome_competicao}','{comp_score}','{part_score}','{result_score}','{Start_date}','{end_date}', "\
                    f"'{cidade_competicao}','{country}','{ps_place}','{rs_place}')"
                contador += 1  
                print (f'Executando Query {contador} executada')
                cursor.execute(sql_update)
        
           # except UniqueViolation:
           #     print("Registro encontrado")
           #     continue     ''' 
            
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error in transction Reverting all other operations of a transction ", error)
    

print("Transaction completed successfully ")
if connection:
    connection.commit()
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")
    
     

## Insertions on other tables

In [ ]:
connection = psycopg2.connect(user="postgres",
                                  password="jujuba11",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db_atletismo",
                                  options="-c search_path=db_atl")


connection.autocommit = True
cursor = connection.cursor()

cursor.execute(f"SELECT cod_competicao  FROM tb_competicao WHERE date_part('year', data_final_competicao) = {year};")
               
list_cod_comp = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

lista_full = []

for compt in list_cod_comp:
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    print (f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('div', attrs = {'class' : 'cpr-result-score'})

    list_competicao = soup.find_all('tbody')[0].find_all('tr')

    for i in list_results:
        try:
            prova = i.find_all('h3')[0].text.split("'s")
            print (f'Prova {prova[0]} na modalidade {prova[1]}')
            score_prova =i.find_all('p')[0].text.split(":")
            cursor.execute(f"INSERT INTO tb_prova (nome_prova, genero_prova) VALUES ('{prova[1]}', '{prova[0]}')")
            
            cursor.execute(f"SELECT cod_prova  FROM tb_prova WHERE nome_prova='{prova[1]}' and genero_prova='{prova[0]}';")
               
            cod_prova = cursor.fetchone()        
            print (f'codigo  da prova {cod_prova[0]}')         
            lista_tb_atleta_competicao = []
            for x in i.find_all('tbody')[0].find_all('tr'):
                score = x.find_all('td')[0].text
                result = x.find_all('td')[1].text
                date = x.find_all('td')[2].text
                cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                lista_full.append((cod_prova[0],score_prova[1].strip(), score.strip(), result.strip(), date.strip(), cod_atleta.strip()))    
                lista_tb_atleta_competicao.append([cod_atleta,result,score])
            atualiza_prova_compet(cursor, cod_prova[0],compt,score_prova[1], date.strip())
            atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao)
            atualiza_atleta_prova_competicao(cursor,cod_prova[0],compt,lista_tb_atleta_competicao)
        
        except IndexError:
            break
        except UniqueViolation:
            print ("Prova ja existe")
            cursor.execute(f"SELECT cod_prova  FROM tb_prova WHERE nome_prova='{prova[1]}' and genero_prova='{prova[0]}';")
            cod_prova = cursor.fetchone()        
            print (f'codigo  da prova {cod_prova[0]}')         
            lista_tb_atleta_competicao = []
            for x in i.find_all('tbody')[0].find_all('tr'):
                try:
                    score = x.find_all('td')[0].text
                    result = x.find_all('td')[1].text
                    date = x.find_all('td')[2].text
                    cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                    lista_full.append((cod_prova[0],score_prova[1].strip(), score.strip(), result.strip(), date.strip(), cod_atleta.strip()))   
                    lista_tb_atleta_competicao.append([cod_atleta,result,score])
                except IndexError:
                    break
            
            atualiza_prova_compet(cursor, cod_prova[0],compt,score_prova[1], date.strip())
            atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao)
            atualiza_atleta_prova_competicao(cursor,cod_prova[0],compt,lista_tb_atleta_competicao)
            continue
if connection:
    connection.commit()
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [10]:

def update_weather (cursor, list_data):
    token_wheter = 'D7RCXA3F55PTG7PNYDPL5K3YB'
    ssl._create_default_https_context = ssl._create_unverified_context

    cidade_competicao = list_data[0][0]
    pais_competicao = list_data[0][1]
    data_prova_competicao = list_data[0][2]
    cursor.execute(f"select count (*) from tb_wheter where  cidade_competicao like '{cidade_competicao}' and "\
               f"pais_competicao like '{pais_competicao}' and data_prova_competicao = '{data_prova_competicao }'")
    list_find_reg = cursor.fetchall() 
    if list_find_reg[0][0] != 1:
        try:
            resp_json = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{cidade_competicao}%2C%20{pais_competicao}/{data_prova_competicao}/{data_prova_competicao}?unitGroup=metric&include=days&key={token_wheter}&contentType=json')
            if resp_json.status_code == 401:
                print (f"Vixe, chegou no limite de hoje, foram {count_query} queries")
            else:
                update_wheter(cursor, data_prova_competicao, pais_competicao, cidade_competicao, json.dumps(resp_json.json()))
        except  JSONDecodeError:
            print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
        except SyntaxError:
            print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
    

In [21]:
pool = Pool(processes=cpu_count()-1)
cursor = connect_db()

cursor.execute(f"select distinct * from cidades where  date_part('year', data_prova_competicao) = 2018")
             
list_data = cursor.fetchall() 

pool.map(update_weather,(cursor, tqdm(list_data)))
         
print ("acabou a atualização do tempo")





  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

TypeError: can't pickle psycopg2.extensions.cursor objects

## Updating climate conditions 

In [6]:
connection = psycopg2.connect(user="postgres",
                                  password="jujuba11",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db_atletismo",
                                  options="-c search_path=db_atl")


connection.autocommit = True
cursor = connection.cursor()
token_wheter = 'D7RCXA3F55PTG7PNYDPL5K3YB'
ssl._create_default_https_context = ssl._create_unverified_context


cursor.execute(f"select distinct * from cidades where  date_part('year', data_prova_competicao) = 2021")
             
list_data = cursor.fetchall()   
for i in list_data:
    cidade_competicao = [i][0][0]
    pais_competicao = [i][0][1]
    data_prova_competicao = [i][0][2]
    cursor.execute(f"select count (*) from tb_wheter where  cidade_competicao like '{cidade_competicao}' and "\
               f"pais_competicao like '{pais_competicao}' and data_prova_competicao = '{data_prova_competicao }'")
    list_find_reg = cursor.fetchall()   

    if list_find_reg[0][0] != 1:
        try:
            resp_json = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{cidade_competicao}%2C%20{pais_competicao}/{data_prova_competicao}/{data_prova_competicao}?unitGroup=metric&include=days&key={token_wheter}&contentType=json')
            if resp_json.status_code == 401:
                print (f"Vixe, chegou no limite de hoje, foram {count_query} queries")
                break
            else:
                count_query += 1
                update_wheter(cursor, data_prova_competicao, pais_competicao, cidade_competicao, json.dumps(resp_json.json()))
        except  JSONDecodeError:
                print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
                continue
                
        except SyntaxError:
                print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
                continue
    else:
        print ("Já existe em tb_wether")
print (lista_erros)
    

nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-07 SUI Aarau
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-09-18 SUI Aarau
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-28 GBR Aberdeen
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-29 GBR Aberdeen
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-12-09 ETH Addis Abeba
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-12-10 ETH Addis Abeba
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-12-11 ETH Addis Abeba
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-11 JPN Ageo
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-01-30 NGR Akure
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-12 NGR Akure
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-13 NGR Akure
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-05 USA AL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-12 US

Inserindo na Tb_wheter -> 2021-06-09 GBR Birmingham
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-10-09 FRA Blois
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-10-10 FRA Blois
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-05 GER Bochum
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-18 GER Bochum
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-02-20 RSA Boksburg
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-13 RSA Boksburg
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-27 RSA Boksburg
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-05 ITA Bolzano
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-17 FRA Bondoufle
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-18 FRA Bondoufle
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-29 FRA Bordeaux
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-20 POR Braga
nao existe e

Inserindo na Tb_wheter -> 2021-09-26 BRA Cascavel
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-16 ESP Castell n
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-09-21 ITA Castelporziano
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-19 ITA Castiglione della Pescaia
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-22 ITA Celle Ligure
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-30 FRA Cergy Pontoise
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-13 FRA Cergy Pontoise
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-16 FRA Cergy Pontoise
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-30 GRE Chania
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-09 RUS Chelyabinsk
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-10 RUS Chelyabinsk
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-01 CHN Chengdu
nao existe em tb_atualizando
Ins

Inserindo na Tb_wheter -> 2021-05-15 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-16 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-30 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-31 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-05 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-11 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-14 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-16 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-17 USA FL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-31 FRA Franconville
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-01 FRA Franconville
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-01 GER Frankfurt
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-02 GER Frankfurt
nao existe em tb_atualizando
Inserindo na Tb_w

Inserindo na Tb_wheter -> 2021-04-17 USA ID
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-06 ESP Igualada
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-19 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-20 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-02 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-09 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-16 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-17 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-23 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-24 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-30 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-01 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-07 USA IL
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-08 USA IL
n

...

Inserindo na Tb_wheter -> 2021-04-24 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-30 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-01 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-07 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-13 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-14 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-15 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-28 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-29 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-01 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-02 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-03 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-20 USA OR
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-21 USA OR
nao exi

Inserindo na Tb_wheter -> 2021-05-23 GER Rehlingen
nao existe em tb_atualizando
Erro na cidade rhus pais DEN
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-27 USA RI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-09 USA RI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-01 USA RI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-02 USA RI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-23 ITA Rieti
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-15 ITA Rieti
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-16 ITA Rieti
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-22 ITA Rieti
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-23 ITA Rieti
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-27 FIN Riihim ki
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-25 ITA Roma
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-0

Erro na cidade Sz kesfeh rv r pais HUN
nao existe em tb_atualizando
Erro na cidade Sz kesfeh rv r pais HUN
nao existe em tb_atualizando
Erro na cidade Sz kesfeh rv r pais HUN
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-08-15 POL Szczecin
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-28 TPE Taipei City
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-29 TPE Taipei City
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-09 CHN Taizhou
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-10 CHN Taizhou
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-21 JPN Takamatsu
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-17 JPN Takamatsu
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-18 JPN Takamatsu
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-19 CHI Talagante
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-20 CHI Talagante
nao existe em tb_atualizan

Inserindo na Tb_wheter -> 2021-03-09 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-13 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-18 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-19 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-20 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-26 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-03-27 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-01 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-02 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-03 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-09 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-10 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-14 USA TX
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-16 USA TX
nao exi

Inserindo na Tb_wheter -> 2021-05-30 POL Warszawa
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-12 POL Warszawa
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-13 POL Warszawa
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-06-26 POL Warszawa
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-18 POL Warszawa
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-05-22 GER Weinheim
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-06 CAN Welland
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-02-20 NZL Wellington
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-07-10 GER Wetzlar
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-02 USA WI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-10 USA WI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-16 USA WI
nao existe em tb_atualizando
Inserindo na Tb_wheter -> 2021-04-17 USA WI
nao existe em tb_atualizan

In [5]:
print (lista_erros)

[['amor n', 'SVK'], ['Bangbu', 'CHN'], ['Bangbu', 'CHN'], ['Bangbu', 'CHN'], ['Dessau', 'GER'], ['Donnas', 'ITA'], ['Donnas', 'ITA'], ['Hastings', 'NZL'], ['Marseille', 'FRA'], ['P rnu', 'EST'], ['rhus', 'DEN'], ['rhus', 'DEN'], ['Setagaya', 'JPN'], ['Setagaya', 'JPN'], ['Sz kesfeh rv r', 'HUN'], ['Sz kesfeh rv r', 'HUN'], ['Veszpr m', 'HUN'], ['Z rich', 'SUI']]


## Updating athletes information 

In [ ]:

cursor = connect_db()

cursor.execute(f"SELECT cod_competicao  FROM tb_competicao WHERE date_part('year', data_final_competicao) = 2012;")
               
list_cod_comp = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

lista_full = []

for compt in list_cod_comp:
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    print (f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('div', attrs = {'class' : 'cpr-result-score'})

    list_competicao = soup.find_all('tbody')[0].find_all('tr')
    
   
    lista_tb_atleta_competicao = []   
    

    for i in list_results:
        
       
            
            for x in i.find_all('tbody')[0].find_all('tr'):
                try:
                        cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                        uri_atleta = x.find_all('td')[3].find_all('span')[0]['data-href']
                        url_atleta =  'https://worldathletics.org' + uri_atleta
                        pais_atleta =  uri_atleta.split('/')[2]
                        nome_atleta =  uri_atleta.split('/')[3]
                        if cod_atleta in lista_tb_atleta_competicao:
                            print ("ja atualizou")
                        else:
                            update_atleta(cursor, cod_atleta,url_atleta,pais_atleta,nome_atleta)
                            lista_tb_atleta_competicao.append(cod_atleta)
                               
                except IndexError:
                    break
                    
            
if connection:
    connection.commit()
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [20]:
def update_age(url):
    return url[0]
   

## Update atlhetes age

In [22]:
%%time
pool = Pool(processes=cpu_count()-1)
cursor = connect_db()

cursor.execute(f"select distinct cod_atleta from db_atl.tb_competicao as T, db_atl.tb_prova_competicao as C, db_atl.tb_prova as P, 
               f"tb_atleta_prova_competicao as A where  T.cod_competicao = C.cod_competicao and C.cod_prova_competicao = A.cod_prova_competicao and
C.cod_prova = P.cod_prova and 
P.nome_prova like '_100 Metres_%'  and
date_part('year', C.data_prova_competicao) = 2019")
             
list_cod_url = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

pool.map(update_age,list_cod_url)


SyntaxError: syntax error at end of input
LINE 1: SELECT url_atleta  FROM tb_atleta limit
                                               ^
